# EIE 1st Year Project - Reference Design 2019

This notebook demonstrates the interface between PYNQ's Processing Systems (PS) and a custom HLS hardware block integrated with PYNQ's base overlay for real-time Video Processing. It implements a Vertical-Edge detector capable of running in >30fps. 

This material can be used for academic purposes only. Any commerical use is prohibited. 

Contact: Alexandros Kouris (a.kouris16@imperial.ac.uk), Ph.D. Candidate, Imperial College London

In [ ]:
from pynq import Overlay, Xlnk
from pynq.overlays.base import BaseOverlay

allocator = Xlnk()

#Load customised version of Base-Overlay,that included the custom hardware block, instead of using BaseOverlay("base.bit")
#------------------------------------------------------------------------------------------------------------------------
#ol = Overlay("eie_v25.bit") #Grayscale, full-frame (33fps)
#ol = Overlay("eie_v25_2.bit") #Grayscale, half-frame (33fps)
ol = Overlay("template.bit") #Vertical Edge-Detector, half-frame (33fps)  - EIE2019

In [2]:
from pynq.lib.video import *

from PIL import Image
sourceFileName = "test.png"
avatar = Image.open(sourceFileName)
avatar.show()

hdmi_in = ol.video.hdmi_in
hdmi_out = ol.video.hdmi_out

hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)

FileNotFoundError: [Errno 2] No such file or directory: 'test.png'

In [3]:
hdmi_in.start()
hdmi_out.start()

NameError: name 'hdmi_in' is not defined

In [4]:
from pynq import MMIO
#filter_reference = MMIO(0x00000000,0x10000)
filter_reference = MMIO(0x83C20000,0x10000)  #MMIO Addresses should always be double-checked when exporting a Vivado Design

In [5]:
import time

start = time.time()

for _ in range(600):
    in_frame = hdmi_in.readframe()
    out_frame = hdmi_out.newframe()
    filter_reference.write(0x10, in_frame.physical_address)   # Get Pointers to memory
    filter_reference.write(0x18, out_frame.physical_address)
    filter_reference.write(0x20, 1280)   # Make sure that the input HDMI signal is set to 1280x720
    filter_reference.write(0x28, 720)
    #filter_reference.write(0x20, in_frame.shape[1]) 
    #filter_reference.write(0x28, in_frame.shape[0])
    filter_reference.write(0x00, 0x01)             # ap_start triggering
    while (filter_reference.read(0) & 0x4) == 0:   # ap_done checking
        pass
    hdmi_out.writeframe(out_frame)

end = time.time()

print(f"600 frames took {end-start} seconds at {600/(end-start)} fps")

600 frames took 18.13244605064392 seconds at 33.08985441479876 fps


In [6]:
hdmi_in.close()    # Don't forget to run this to free memory
hdmi_out.close()   # NEVERFORGET NEVERFORGET NEVERFORGET :p 